## Setup

In [2]:
import os, json, time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from openai import OpenAI
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [4]:
make_criteria_prompt = open("prompts/criteria.prompt", "r").read()
make_probabilites_prompt = open("prompts/make_probabilities.prompt", "r").read()

## Ask News

In [4]:
from asknews_sdk import AskNewsSDK
ask = AskNewsSDK(
    client_id=os.getenv('ASKNEWS_ID'),
    client_secret=os.getenv('ASKNEWS_SECRET')
)

In [5]:
answer_subquestion_prompt = open("prompts/ask_news/answer_sub.prompt", "r").read()
make_subquestion_prompt = open("prompts/ask_news/make_subquestion.prompt", "r").read()

In [6]:
def get_sources(query, end_date:datetime):

    response = ask.news.search_news(
        query=query,
        n_articles=20,
        return_type="dicts",
        method="nl",
        end_timestamp=int(end_date.timestamp()),
        historical=True
    )

    output = []
    for x in response.as_dicts:
        entry = {
            "title": x.title,
            "summary": x.summary,
            "pub_date": x.pub_date.strftime("%Y-%m-%d"),
            "source": x.source_id
        }
        output.append(entry)

    return output

In [7]:
def answer_subquestion(question:str, subquestion:str, date:datetime):

    prompt = answer_subquestion_prompt.format(question=question, subquestion=subquestion)
    sources = get_sources(question, date)

    payload = ""
    payload += f"ORGINAL QUESTION: {question}"
    payload += f"\nSECONDARY QUESTION: {subquestion}"
    payload += "\n\nRELEVANT WEB SOURCES:"
    for x in sources:
        prompt += f"\n\nTITLE: {x['title']}\nPUB DATE: {x['pub_date']}\nSOURCE: {x['source']}\nSUMMARY: {x['summary']}"
    prompt += "\n\nANSWER TO SECONDARY QUESTION:\n"
    response = openai_client.chat.completions.create(
        seed=42,
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}])
    return response.choices[0].message.content

In [9]:
def make_forecast(
        question:str, 
        date:datetime=None,
        background:str=None,
        criteria:str=None, 
        fine_print:str=None):

    if date is None:
        date = datetime.now()

    if criteria is None:
        prompt = make_criteria_prompt.format(question=question)
        response = openai_client.chat.completions.create(
            seed=42,
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
        )
        criteria = response.choices[0].message.content

    # get relevant sources
    sources = get_sources(question, date)

    # decompose question into subquestions
    payload = ""
    payload += f"QUESTION:\n{question}"
    payload += f"\n\nTODAY'S DATE:\n{date.strftime('%Y-%m-%d')}"
    payload += f"\n\nRESOLUTION CRITERIA:\n{criteria}"
    
    if fine_print:
        payload += f"\n\nADDITIONAL CRITERIA:\n{fine_print}"

    if background:
        payload += f"\n\nBACKGROUND:\n{background}"

    payload += '\n\nRELEVANT WEB SOURCES:'
    for x in sources:
        payload += f"\n\nTITLE: {x['title']}\nPUB DATE: {x['pub_date']}\nSOURCE: {x['source']}\nSUMMARY: {x['summary']}"

    response = openai_client.chat.completions.create(
        seed=42,
        model="gpt-4o",
        messages=[
            {"role": "user", "content": make_subquestions_prompt},
            {"role": "user", "content": payload}],
        response_format={ "type": "json_object" }
    )

    subquestions = response.choices[0].message.content
    subquestions = json.loads(subquestions)
    assert "subquestions" in subquestions
    subquestions = subquestions["subquestions"]

    sub_values = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {}
        for subquestion in subquestions:
            future = executor.submit(answer_subquestion, question, subquestion, date)
            futures[future] = subquestion
        for future in as_completed(futures):
            value = future.result()
            subquestion = futures[future]
            sub_values.append((subquestion, value))
    
    temp = []
    for subquestion, value in sub_values:
        if 'UNCLEAR' in value:
            temp.append((subquestion, 'UNCLEAR'))
        elif 'YES' in value:
            temp.append((subquestion, 'YES'))
        elif 'NO' in value:
            temp.append((subquestion, 'NO'))
    sub_values = temp

    if sub_values:
        payload += "\n\nRELEVANT SUBQUESTIONS:"
        for subquestion, value in sub_values:
            payload += f"\n\nSUBQUESTION: {subquestion}\nANSWER: {value}"

    print(payload)
    print('-'*80)

    response = openai_client.chat.completions.create(
        seed=42,
        model="gpt-4o",
        messages=[
            {"role": "user", "content": make_probabilites_prompt},
            {"role": "user", "content": payload}],
    )

    final_response = response.choices[0].message.content
    print(final_response)

In [ ]:
make_forecast(
    question = "Will the Real-time Sahm Rule Recession Indicator increase Year-over-Year for Nov 2024?",
    background = """According to FRED: > Sahm Recession Indicator signals the start of a recession when the three-month moving average of the national unemployment rate (U3) rises by 0.50 percentage points or more relative to the minimum of the three-month averages from the previous 12 months. This indicator is based on "real-time" data, that is, the unemployment rate (and the recent history of unemployment rates) that were available in a given month. The BLS revises the unemployment rate each year at the beginning of January, when the December unemployment rate for the prior year is published. Revisions to the seasonal factors can affect estimates in recent years. Otherwise the unemployment rate does not revise.""",
    criteria="This question will resolve based on reporting by the St. Louis Fed here:  https://fred.stlouisfed.org/series/SAHMREALTIME. The reported number for Nov 2024 must be greater than the reported for Nov 2023.",
    fine_print="The question will resolve according to the first value published for the period in question, later updates or revisions will be immaterial."
)

In [ ]:
make_forecast(
    "Will Prince Tom Iseghohi win the September 21, 2024 Edo state gubernatorial election in Nigeria?",
    date=datetime(2024, 9, 19),
    criteria="This question resolves as Yes if Prince Tom Iseghohi is the next governor of Edo State in Nigeria, based on the results of the September 21, 2024 gubernatorial election according to the Independent National Electoral Commission of Nigeria or credible media reports on the election results.",
    fine_print="Although Nigeria typically counts its votes within a few days, if there is no declared winner before October 1, 2024, this question will be annulled.",
    background="""Prince Tom Iseghohi is the Action Alliance's nominee for governor of Edo. The former Managing Director at Transnational Corp of Nigeria, Iseghohi was acquitted of money laundering charges in 2017.  Nigeria's Vanguard News in April 2024 named Iseghohi the frontrunner in the race, adding that "is not only the fastest growing party in Nigeria but also the party to beat in the forthcoming September 21 governorship election in the state.\""""
)

## Perplexity

In [5]:
from openai import OpenAI
perplexity_client = OpenAI(
    api_key=os.environ["PERPLEXITY_KEY"], 
    base_url="https://api.perplexity.ai")

In [6]:
make_background_prompt = open("prompts/perplexity/make_background.prompt", "r").read()
answer_subquestion_prompt = open("prompts/perplexity/answer_sub.prompt", "r").read()
make_subquestions_prompt = open("prompts/perplexity/make_sub.prompt", "r").read()

In [7]:
def make_background(question:str):

    messages = [{'role':'system','content':make_background_prompt},
                {'role':'user','content':question}]
    response = perplexity_client.chat.completions.create(
        seed=42,
        model="llama-3.1-sonar-large-128k-online",
        messages=messages
    )
    return response.choices[0].message.content

In [8]:
def answer_subquestion(subquestion:str):

    messages = [{'role':'system','content':answer_subquestion_prompt},
                {'role':'user','content':subquestion}]
    response = perplexity_client.chat.completions.create(
        seed=42,
        model="llama-3.1-sonar-large-128k-online",
        messages=messages
    )
    return response.choices[0].message.content

In [9]:
import re
def process_forecast_probabilty(forecast_text: str):
    """
    Extract the forecast probability from the forecast text and clamp it between 1 and 99.
    """
    matches = re.findall(r"(\d+)%", forecast_text)
    if matches:
        # Return the last number found before a '%'
        number = int(matches[-1])
        number = min(99, max(1, number))  # clamp the number between 1 and 99
        return number
    else:
        return None

In [10]:
def make_forecast(
        question:str, 
        date:datetime=None,
        background:str=None,
        criteria:str=None, 
        fine_print:str=None):
    
    if date is None:
        date = datetime.now()

    if criteria is None:
        prompt = make_criteria_prompt.format(question=question)
        response = openai_client.chat.completions.create(
            seed=42,
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
        )
        criteria = response.choices[0].message.content
    
    # get perplexity background
    extra_background = make_background(question)

    # decompose question into subquestions
    payload = ""
    payload += f"QUESTION:\n{question}"
    payload += f"\n\nTODAY'S DATE:\n{date.strftime('%Y-%m-%d')}"
    payload += f"\n\nRESOLUTION CRITERIA:\n{criteria}"
    
    if fine_print:
        payload += f"\n\nADDITIONAL CRITERIA:\n{fine_print}"

    if background:
        payload += f"\n\nBACKGROUND:\n{background}"

    payload += f'\n\nADDITIONAL BACKGROUND:\n{extra_background}'

    print(payload)

    response = openai_client.chat.completions.create(
        seed=42,
        model="o1-preview",
        messages = [
            {"role": "user", "content": make_subquestions_prompt},
            {"role": "user", "content": payload}]
    )

    subquestions = response.choices[0].message.content
    subquestions = subquestions.split('\n')
    subquestions = [x.strip() for x in subquestions]
    subquestions = [x for x in subquestions if x]

    display(subquestions)

    # make answers to each subquestion
    sub_values = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {}
        for subquestion in subquestions:
            future = executor.submit(answer_subquestion, subquestion)
            futures[future] = subquestion
        for future in as_completed(futures):
            value = future.result()
            subquestion = futures[future]
            sub_values.append((subquestion, value))

    # make final probability
    payload += "\n\nRELEVANT SUBQUESTIONS:"
    for subquestion, value in sub_values:
        payload += f"\n\nSUBQUESTION: {subquestion}\nANSWER: {value}"

    response = openai_client.chat.completions.create(
        seed=42,
        model="o1-preview",
        messages=[
            {"role": "user", "content": make_probabilites_prompt},
            {"role": "user", "content": payload}],
    )

    final_response = response.choices[0].message.content

    # parse final_response to extract probability
    probability = process_forecast_probabilty(final_response)

    return (final_response, probability)

In [11]:
resp, prob = make_forecast(
    question = "Will the Real-time Sahm Rule Recession Indicator increase Year-over-Year for Nov 2024?",
    background = """According to FRED: > Sahm Recession Indicator signals the start of a recession when the three-month moving average of the national unemployment rate (U3) rises by 0.50 percentage points or more relative to the minimum of the three-month averages from the previous 12 months. This indicator is based on "real-time" data, that is, the unemployment rate (and the recent history of unemployment rates) that were available in a given month. The BLS revises the unemployment rate each year at the beginning of January, when the December unemployment rate for the prior year is published. Revisions to the seasonal factors can affect estimates in recent years. Otherwise the unemployment rate does not revise.""",
    criteria="This question will resolve based on reporting by the St. Louis Fed here:  https://fred.stlouisfed.org/series/SAHMREALTIME. The reported number for Nov 2024 must be greater than the reported for Nov 2023.",
    fine_print="The question will resolve according to the first value published for the period in question, later updates or revisions will be immaterial."
)

print(resp)
print(prob)

QUESTION:
Will the Real-time Sahm Rule Recession Indicator increase Year-over-Year for Nov 2024?

TODAY'S DATE:
2024-10-16

RESOLUTION CRITERIA:
This question will resolve based on reporting by the St. Louis Fed here:  https://fred.stlouisfed.org/series/SAHMREALTIME. The reported number for Nov 2024 must be greater than the reported for Nov 2023.

ADDITIONAL CRITERIA:
The question will resolve according to the first value published for the period in question, later updates or revisions will be immaterial.

BACKGROUND:
According to FRED: > Sahm Recession Indicator signals the start of a recession when the three-month moving average of the national unemployment rate (U3) rises by 0.50 percentage points or more relative to the minimum of the three-month averages from the previous 12 months. This indicator is based on "real-time" data, that is, the unemployment rate (and the recent history of unemployment rates) that were available in a given month. The BLS revises the unemployment rate ea

['Did the Real-time Sahm Rule Recession Indicator for November 2023 exceed 0.10%?',
 'Was the national unemployment rate in November 2024 higher than in November 2023?',
 'Has the three-month moving average of the national unemployment rate for November 2024 increased compared to November 2023?',
 'Did the national unemployment rate rise by at least 0.50 percentage points in the three months leading up to November 2024 compared to the previous 12-month minimum?',
 'Have there been any significant revisions to the unemployment rate data for November 2024 after its initial release?',
 'Has the unemployment rate consistently increased from August 2024 to November 2024?',
 'Have economic indicators such as household income or consumer spending declined in the months leading up to November 2024?',
 'Has the labor market shown signs of weakening in the months leading up to November 2024?',
 'Were there any major economic policy changes affecting unemployment between November 2023 and Novembe

**(a) The time left until the outcome to the question is known.**
- **Approximately 1.5 weeks** remain until the Real-time Sahm Rule Recession Indicator for November 2024 is published.

**(b) What the outcome would be if nothing changed.**
- **If nothing changes**, the Real-time Sahm Rule Recession Indicator would **remain at or slightly above its current level of 0.57%**, maintaining an increase year-over-year compared to November 2023.

**(c) What you would forecast if there was only a quarter of the time left.**
- **With only a quarter of the time left (approximately 3-4 weeks)**, I would forecast a **high probability (around 85%)** that the indicator will continue its upward trend, given the current weakening labor market conditions and the existing upward trend in the indicator.

**(d) What you would forecast if there was 4x the time left.**
- **With 4 times the time left (approximately 6 weeks)**, there is more room for economic conditions to change. Factors such as potential pol

## 